In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss



In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

14.379764080047607


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

idxList = []
for i in allTextToxicTrain.keys():
    #allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]
    idxList = np.unique(np.append(idxList, allTextToxicTrain[i]))
allTextToxicTrain = [str(train['cleanWordsAsText'][j]) for j in idxList]

In [ ]:
###### GET THE BEST WORDS IN ALL CORPUS #####

topNFeatures = 1000
Nfolds = 5

#Neutral data

tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxicTrain)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]

#Toxic data
selectedWordsTrainToxic = []
selectedWordsTrain = []

tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextToxicTrain)
featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]
selectedWordsTrainToxic = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]

# Get words
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        words = [selectedWordsTrainToxic[x]] + [selectedWordsTrainNoToxic[x]]
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique(words)))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]


In [ ]:
allTrainText = [txt if txt is not np.nan else '' for txt in train['cleanWordsAsText']]

X = allTrainText
yBinary = MultiLabelBinarizer().fit_transform(y)

kf = KFold(n_splits=Nfolds, random_state=True)
kf.get_n_splits(allTrainText)

In [ ]:
# PREDICCION MULTICLASS Linear SVC

meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n Linear SVC")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

In [ ]:
# PREDICCION MULTICLASS Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(DecisionTreeClassifier()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n Decission tree")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

In [ ]:
# PREDICCION MULTICLASS SGD
from sklearn import linear_model
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(linear_model.SGDClassifier()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n SGD")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

In [ ]:
# PREDICCION MULTICLASS NB
from sklearn.naive_bayes import MultinomialNB
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(MultinomialNB()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n NB")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))